This notebook includes all the code to preprocess datasets for the experiments in the paper. There will be two main parts: the first half is for row pairs preparation, while the second half is for attribute pairs preparation.

# Row Pairs Preparation

## Training & Validation Set

The magellan datasets and wdc dataset need different preparation steps. We will first prepare the magellan datasets.

In [ ]:
import os
import pandas as pd
import json
from autogluon.tabular import TabularPredictor

### Magellan Datasets

In [ ]:
magellan_dirs = {
    'amgo': 'raw/amazon_google',
    'beer': 'raw/beer', 'dbac': 'raw/dblp_acm',
    'dbgo': 'raw/dblp_scholar', 'foza': 'raw/fodors_zagat',
    'itam': 'raw/itunes_amazon', 'waam': 'raw/walmart_amazon',
}

magellan_rename_columns = {
    'amgo': ['id', 'name', 'manufacturer', 'price'],
    'beer': ['id', 'name', 'factory', 'style', 'ABV'], 'dbac': ['id', 'title', 'authors', 'venue', 'year'],
    'dbgo': ['id', 'title', 'authors', 'venue', 'year'], 'foza': ['id', 'name', 'address', 'city', 'phone', 'type', 'class'],
    'itam': ['id', 'name', 'artist', 'album', 'genre', 'price', 'copyright', 'time', 'released'],
    'waam': ['id', 'name', 'category', 'brand', 'modelno', 'price'],
}

magellan_drop_columns = {
    'amgo': ['manufacturer'], 'beer': [], 'dbac': [], 'dbgo': [], 'foza': [], 'itam': [],
    'waam': ['category', 'brand'],
}

In [ ]:
def analyze_reasoning(text, remove_answer=False):
    if pd.isna(text):
        return (None, "")
    
    label = None
    if "Answer: Yes." in text:
        label = 1
    elif "Answer: No." in text:
        label = 0
        
    if remove_answer:
        processed_text = text.replace("Answer: Yes.", "").replace("Answer: No.", "")
    else:
        processed_text = text
            
    return label, processed_text.strip()

def filter_valid_reasoning(df):
    valid_mask = (
        df['reasoning_text'].notna() & 
        (df['reasoning_text'] != '') & 
        (~df['reasoning_text'].str.contains('Error code: 400', na=False)) 
    )
    filtered_df = df[valid_mask].reset_index(drop=True)
    removed_count = len(df) - len(filtered_df)
    print(f"Filtered {removed_count} samples with empty reasoning_text")
    
    return filtered_df

def add_reasoning_to_df(df, reasoning_dict, remove_answer=False):
    reasoning_texts = []
    for _, row in df.iterrows():
        key = (row['ltable_id'], row['rtable_id'])
        reasoning_text = reasoning_dict.get(key, "")
        reasoning_texts.append(reasoning_text)
    df['reasoning_text'] = reasoning_texts
    
    df = filter_valid_reasoning(df)

    results = df['reasoning_text'].apply(analyze_reasoning)
    df['answer_label'] = [result[0] for result in results]
    df['reasoning_text'] = [result[1] for result in results]

    yes_count = (df['answer_label'] == 1).sum()
    no_count = (df['answer_label'] == 0).sum()
    unclassified = df['answer_label'].isna().sum()
    print(f"Total rows: {len(df)}")
    print(f"'Yes' (1) count: {yes_count}")
    print(f"'No' (0) count: {no_count}")
    print(f"Unclassified data count: {unclassified}")
    
    return df

def load_reasoning_data(dataset_name, split):
    reasoning_file = f'reasoning/{dataset_name}/{split}_results.jsonl'
    if not os.path.exists(reasoning_file):
        print(f"Warning: Reasoning file {reasoning_file} not found")
        return {}
    
    reasoning_dict = {}
    with open(reasoning_file, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line.strip())
            custom_id = data['custom_id']
            ltable_id, rtable_id = custom_id.split('_')
            reasoning_dict[(int(ltable_id), int(rtable_id))] = data['response']
    
    return reasoning_dict

In [ ]:
def merge_with_id(tableA, tableB, id_pairs):
    left_merged = pd.merge(tableA, id_pairs, left_on='id', right_on='ltable_id')
    left_right_merged = pd.merge(left_merged, tableB, left_on='rtable_id', right_on='id', suffixes=('_l', '_r'))
    left_right_merged.drop(columns=['id_l', 'id_r'], inplace=True)
    return left_right_merged

In [ ]:
def prepare_magellan_row_pairs(dirs: dict, rename_columns: dict, drop_columns: dict, remove_answer: bool):
    for d_name in dirs:
        tableA = pd.read_csv(os.path.join(dirs[d_name], 'tableA.csv'))
        tableB = pd.read_csv(os.path.join(dirs[d_name], 'tableB.csv'))
        tableA.columns = rename_columns[d_name]
        tableB.columns = rename_columns[d_name]
        tableA.drop(columns=drop_columns[d_name], inplace=True)
        tableB.drop(columns=drop_columns[d_name], inplace=True)

        train_id_pairs = pd.read_csv(os.path.join(dirs[d_name], 'train.csv'))
        valid_id_pairs = pd.read_csv(os.path.join(dirs[d_name], 'valid.csv'))
        test_id_pairs = pd.read_csv(os.path.join(dirs[d_name], 'test.csv'))
        train_df = merge_with_id(tableA, tableB, train_id_pairs)
        valid_df = merge_with_id(tableA, tableB, valid_id_pairs)
        test_df = merge_with_id(tableA, tableB, test_id_pairs)
        
        train_reasoning = load_reasoning_data(d_name, 'train')
        valid_reasoning = load_reasoning_data(d_name, 'valid')
        test_reasoning = load_reasoning_data(d_name, 'test')
        train_df = add_reasoning_to_df(train_df, train_reasoning, remove_answer)
        valid_df = add_reasoning_to_df(valid_df, valid_reasoning, remove_answer)
        test_df = add_reasoning_to_df(test_df, test_reasoning, remove_answer)
        train_df.drop(columns=['ltable_id', 'rtable_id'], inplace=True)
        valid_df.drop(columns=['ltable_id', 'rtable_id'], inplace=True)
        test_df.drop(columns=['ltable_id', 'rtable_id'], inplace=True)

        if not os.path.exists(f'prepared/{d_name}'):
            os.makedirs(f'prepared/{d_name}')
        train_df.to_csv(f'prepared/{d_name}/train.csv', index=False)
        valid_df.to_csv(f'prepared/{d_name}/valid.csv', index=False)
        test_df.to_csv(f'prepared/{d_name}/test.csv', index=False)
        

In [ ]:
prepare_magellan_row_pairs(
    magellan_dirs, 
    magellan_rename_columns, 
    magellan_drop_columns,
    remove_answer=False)

# AutoML Predictions

In [ ]:
def prepare_automl_predictions():
    dataset_names = ['amgo', 'beer', 'dbac', 'dbgo', 'foza', 'itam', 'waam']
    for name in dataset_names:
        train_df = pd.read_csv(f'prepared/{name}/train.csv')
        valid_df = pd.read_csv(f'prepared/{name}/valid.csv')

        feature_columns = [col for col in train_df.columns if col not in ['reasoning_text', 'answer_label', 'label']]
        train_automl = train_df[feature_columns + ['label']].copy()
        valid_automl = valid_df[feature_columns + ['label']].copy()

        predictor = TabularPredictor(label='label').fit(train_data=train_automl, tuning_data=valid_automl, verbosity=-1)
        train_preds = predictor.predict(train_automl)
        train_preds_proba = predictor.predict_proba(train_automl)
        valid_preds = predictor.predict(valid_automl)
        valid_preds_proba = predictor.predict_proba(valid_automl)
        train_preds_df = pd.DataFrame({'prediction': train_preds, 'proba_0': train_preds_proba[0], 'proba_1': train_preds_proba[1]})
        valid_preds_df = pd.DataFrame({'prediction': valid_preds, 'proba_0': valid_preds_proba[0], 'proba_1': valid_preds_proba[1]})

        if not os.path.exists(f'automl/{name}'):
            os.makedirs(f'automl/{name}')
        train_preds_df.to_csv(f'automl/{name}/train_preds.csv', index=False)
        valid_preds_df.to_csv(f'automl/{name}/valid_preds.csv', index=False)

In [ ]:
prepare_automl_predictions()

No path specified. Models will be saved in: "AutogluonModels/ag-20250408_183719"
No path specified. Models will be saved in: "AutogluonModels/ag-20250408_184026"
